In [10]:
import pandas as pd 
import numpy as np 
import spacy 
import gensim

In [21]:
df = pd.read_csv('../base/pre_processamento.csv', index_col=0)


In [12]:
df 

,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
dataPublicada,,,,,,,,,,,
"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']"
"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim']
"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking', 'própria']"
"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[]
"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']"
...,...,...,...,...,...,...,...,...,...,...,...
"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']"
"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['porque', 'brg', 'tá', 'diminuído', 'limite',..."
"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']"


*1* - Baixe o modelo treinado de word embedding "CBOW 50 dimensões" do site do NILC (http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc)

2 - Coloque o arquivo extraído "cbow_s50.txt" na pasta com o nome "base" do repositório


In [13]:
cbow = '../base/cbow_s50.txt'


In [14]:
from scipy.spatial.distance import cosine

from gensim.models import KeyedVectors

model_cbow = KeyedVectors.load_word2vec_format(cbow)

In [15]:
# Testando o word2vec
wordvec_word1 = model_cbow['banco']

wordvec_word1

array([ 1.81041e-01,  1.07700e-01, -1.04667e-01,  2.43361e-01,
        6.06380e-02,  3.92829e-01, -3.33944e-01, -3.81778e-01,
        1.42200e-01,  8.59360e-02, -1.16615e-01,  3.95722e-01,
       -6.12684e-01, -7.68980e-02,  3.34396e-01,  8.11270e-02,
       -5.17700e-02, -3.21950e-01, -6.91509e-01, -3.31210e-01,
       -5.43213e-01,  6.09881e-01,  2.43700e-01,  3.73240e-02,
        1.16518e-01,  1.78859e-01, -3.78839e-01,  1.27430e-01,
        1.94497e-01,  7.32000e-04,  3.14395e-01, -2.04550e-01,
        5.34431e-01, -5.55100e-03,  3.52343e-01, -4.92000e-02,
       -1.38384e-01,  2.31630e-02, -3.40013e-01,  5.00201e-01,
       -1.14170e-02, -1.29925e-01, -6.12800e-03, -1.80481e-01,
        1.99391e-01,  1.37645e-01, -7.66434e-01, -2.26784e-01,
       -6.16110e-02,  9.05920e-02], dtype=float32)

In [30]:
# Substituindo todos os elementos vazios da coluna texto_tokenizado por NaN
df['texto_tokenizado'].replace('[]', np.nan, inplace=True)


In [22]:
df

,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
dataPublicada,,,,,,,,,,,
"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']"
"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim']
"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking', 'própria']"
"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[]
"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']"
...,...,...,...,...,...,...,...,...,...,...,...
"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']"
"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['porque', 'brg', 'tá', 'diminuído', 'limite',..."
"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']"


In [25]:
# Removendo todos os NaN
df = df.dropna()
df

,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
dataPublicada,,,,,,,,,,,
"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']"
"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim']
"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking', 'própria']"
"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']"
"""2022-03-03 15:35:00""",rafael_miller,Vamos ajudar o Brasil na principal categoria d...,POSITIVE,comentário,0,41,https://www.instagram.com/p/CapoqJ3Ne7a/#17972...,0,0,Vamos ajudar o Brasil na principal categoria d...,"['vamos', 'ajudar', 'brasil', 'principal', 'ca..."
...,...,...,...,...,...,...,...,...,...,...,...
"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']"
"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['porque', 'brg', 'tá', 'diminuído', 'limite',..."
"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']"


In [26]:
def create_sentence_vector(model, cleaned_df):
    sentence_table = []
    for sentence in cleaned_df['texto_tokenizado']:
        word_vectors = [model[word] for word in sentence if word in model]
        if len(word_vectors) > 0:
            sentence_vector = sum(word_vectors) / len(word_vectors)
        else:
            sentence_vector = [None] * 100  # Cria uma lista de 100 elementos None
        sentence_table.append((sentence, *sentence_vector[:50]))  # Adiciona apenas os primeiros 50 elementos do vetor

    column_labels = ['Frase']
    for i in range(50):
        column_labels.append(f'Vetor{i+1}')
    df_vec = pd.DataFrame(sentence_table, columns=column_labels)

    cleaned_df["sentimentoNumerico"] = cleaned_df["sentimento"].replace({'NEGATIVE': -1, 'POSITIVE': 1, 'NEUTRAL': 0})

    # Definir o índice do DataFrame df_vec como o mesmo índice de df_processada['sentimentoNumerico']
    df_vec.set_index(cleaned_df["sentimentoNumerico"].index, inplace=True)

    df_vec['sentimento'] = cleaned_df["sentimentoNumerico"]
    df_vec = df_vec.dropna()

    return df_vec

In [27]:
df_vec = create_sentence_vector(model_cbow, df)
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
dataPublicada,,,,,,,,,,,,,,,,,,,,,
"""2022-03-04 08:36:00""","['minuto', 'touro', 'ouro']",0.194111,-0.086163,0.150576,-0.034733,-0.066786,0.122470,0.027893,0.064956,0.005728,...,0.046650,-0.142142,0.131115,0.063196,0.025213,0.094671,-0.001749,-0.002428,0.263365,1
"""2022-03-03 21:10:00""",['sim'],0.109873,-0.062076,0.187268,-0.213734,0.046350,0.118452,-0.025577,-0.149704,0.143837,...,0.031963,-0.051879,0.235329,-0.005905,0.000182,-0.029570,-0.064395,-0.025853,0.185357,0
"""2022-03-03 20:54:00""","['queria', 'saber', 'banking', 'própria']",0.121342,-0.183903,0.268051,-0.031265,0.003071,0.199323,-0.008417,0.046410,0.046311,...,0.029866,-0.076881,0.146361,0.077992,-0.022279,0.108679,-0.068519,-0.027490,0.130485,1
"""2022-03-03 16:11:00""","['palmas', 'palmas', 'palmas', 'palmas']",0.215186,-0.092701,0.254579,-0.083714,0.099877,0.371233,0.050387,-0.127568,0.118620,...,-0.090724,0.021903,0.221911,0.072890,0.119573,-0.004307,-0.091760,0.027266,0.144531,0
"""2022-03-03 15:35:00""","['vamos', 'ajudar', 'brasil', 'principal', 'ca...",0.208002,-0.119283,0.219406,-0.040021,0.008186,0.211272,0.019965,0.025391,0.058953,...,-0.018090,-0.062190,0.159715,0.074844,0.002553,0.096084,-0.036452,0.003484,0.147662,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""2022-11-30 07:48:00""","['excelente', 'explicação']",0.156762,-0.134906,0.226852,-0.045667,0.064145,0.179836,0.001532,-0.080106,-0.059974,...,-0.077624,0.021257,0.139206,0.068294,0.009093,0.062578,-0.032271,-0.032976,0.160159,1
"""2022-11-30 08:42:00""","['porque', 'brg', 'tá', 'diminuído', 'limite',...",0.166784,-0.125591,0.187974,-0.061618,0.001240,0.180365,0.004502,0.012454,0.067296,...,0.006491,-0.059519,0.148439,0.087409,0.034425,0.070620,-0.056318,0.015074,0.171040,-1
"""2022-11-30 11:27:00""","['atendam', 'telefone', 'amor', 'deus']",0.136519,-0.113854,0.161438,-0.110917,0.034236,0.139376,0.029379,-0.063767,0.037123,...,0.000793,-0.079159,0.139690,0.086531,0.027053,0.063675,-0.031798,0.055869,0.187216,1


In [29]:
# Exportando o csv com o resultado da aplicação do word2vec com os vetores pré treinados 
df_vec.to_csv('..\\output\\word2vec_cbow.csv',encoding='utf-8', index=False, header=True, )